
# AWSDX -> sdx-end-to-end-tests  
#### (derived from) Functional Test 3.2.1 - Local bridge with Shared NICs

This Jupyter notebook will allow you to create VMs on different sites and worker nodes consistent with requirements for test 3.2.1 for testing Shared NIC with a local bridge.

## Step 1:  Configure the Environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../fablib_api/configure_environment/configure_environment.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

**This only needs to be done once.**

## Step 2: Import the FABlib Library


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

## Step 3: Check your existing slices

Since testing can get confusing, check what slices you actually have. It may print nothing if you have no active slices.

In [ ]:
try:
    for slice in fablib.get_slices():
        print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

## Step 4: Create the Slice

The following creates two nodes with a shared NIC each. This should be run all worker nodes regardless of type.

Two nodes with one NIC component each are created on different workers.  This example uses components of model `NIC_Basic` which are SR-IOV Virtual Function on a 100 Gpbs Mellanox ConnectX-6 PCI device. The VF is accessed by the node via PCI passthrough. 

**Be sure to try different combinations of workers**

In [ ]:
from datetime import datetime
from dateutil import tz

name1='Node1'
nic1_name='SharedNIC1'

name2='Node2'
nic2_name='SharedNIC2'

network_name='l2-bridge'

site='FIU'

# since all workers have a standard naming scheme, you can just change the worker
# to move from worker to worker
#worker1=f'{site.lower()}-w1.fabric-testbed.net'
#worker2=f'{site.lower()}-w2.fabric-testbed.net'

cores=10
ram=20
disk=50

slice_name=f"AWSDX end-to-end-tests {site} {datetime.now().strftime('%Y-%m-%d-%H-%M')}"

### List of the VM images

Available images on FABRIC Testbed

```
+------------------------------+
| Name                         |
+------------------------------+
| default_debian_11            |
| default_debian_12            |
| default_rocky_8              |
| default_rocky_9              |
| default_ubuntu_20            |
| default_ubuntu_22            |
| default_ubuntu_24            |
| docker_rocky_8               |
| docker_rocky_9               |
| docker_ubuntu_20             |
| docker_ubuntu_22             |
| docker_ubuntu_24             |
+------------------------------+
```

In [ ]:
try:
    #Create Slice
    print(f'Creating slice {slice_name}')
    slice = fablib.new_slice(name=slice_name)

    # Node1
    node1 = slice.add_node(name=name1, site=site, cores=cores, ram=ram, disk=disk, image="docker_rocky_8")
    iface1 = node1.add_component(model='NIC_Basic', name=nic1_name).get_interfaces()[0]
    
    node2 = slice.add_node(name=name2, site=site, cores=cores, ram=ram, disk=disk, image="default_debian_12")
    iface2 = node2.add_component(model='NIC_Basic', name=nic2_name).get_interfaces()[0]
    
    # Network
    net1 = slice.add_l2network(name=network_name, interfaces=[iface1, iface2])
 
    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")

## Step 5: Observe the Slice's Attributes

### Print the slice 

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

## Print the Node List

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)

    print(f"{slice.list_nodes()}")
except Exception as e:
    print(f"Exception: {e}")

## Print the Node Details

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

## Print the Interfaces

You should see 2 interfaces.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    
    print(f"{slice.list_interfaces()}")
except Exception as e:
    print(f"Exception: {e}")

## Step 6: Configure interfaces, test reachability

##  Configure IP Addresses

### Pick a Subnet

Create a subnet and list of available IP addresses. You can use either IPv4 or IPv6 subnets and addresses.

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:
    subnet = IPv4Network("192.168.1.0/24")
    available_ips = list(subnet)[1:]
except Exception as e:
    print(f"Exception: {e}")

### Configure Node1

Get the node and the interface you wish to configure.  You can use `node.get_interface` to get the interface that is connected to the specified network.  Then `pop` an IP address from the list of available IPs and call `iface.ip_addr_add` to set the IP and subnet.  

Optionally, use the `node.execute()` method to show the results of adding the IP address.

In [ ]:
try:
    node1 = slice.get_node(name=name1)        
    node1_iface = node1.get_interface(network_name=network_name) 
    node1_addr = available_ips.pop(0)
    node1_iface.ip_addr_add(addr=node1_addr, subnet=subnet)
    
    stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')
    print (stdout)
    
except Exception as e:
    print(f"Exception: {e}")

### Configure Node2

Repeat the steps to add the next available IP to the second node.

In [ ]:
try:
    node2 = slice.get_node(name=name2)        
    node2_iface = node2.get_interface(network_name=network_name)  
    node2_addr = available_ips.pop(0)
    node2_iface.ip_addr_add(addr=node2_addr, subnet=subnet)
    
    stdout, stderr = node2.execute(f'ip addr show {node2_iface.get_os_interface()}')
    print (stdout)
    
except Exception as e:
    print(f"Exception: {e}")

### Test reachability

Test ping between interfaces, observe successful output.


In [ ]:
try:
    node1 = slice.get_node(name=name1)        

    stdout, stderr = node1.execute(f'ping -c 5 {node2_addr}')
    print (stdout)
    print (stderr)
    
except Exception as e:
    print(f"Exception: {e}")

## Step 7: Execute sdx-end-to-end-tests

- https://github.com/atlanticwave-sdx/sdx-end-to-end-tests.git
- https://github.com/atlanticwave-sdx/sdx-deployment/issues/102
- https://github.com/atlanticwave-sdx/sdx-deployment/issues/106

### Select the node 

- node1 : created with the docker_rocky_8 image that already has docker installed. No further action needed for docker installation
- node2 : created with the default_debian_12 image.  Install docker -> https://docs.docker.com/engine/install/debian/


In [ ]:
node = node1
#node = node2

if node == node1:
    print(f'--- Use Node: {node}')
    commands = list()
    commands.append(f'sudo dnf install -y jq')

elif node == node2: 
    print(f'--- Use Node: {node}')
    commands = list()
    commands.append(f'sudo apt-get update')
    commands.append(f'sudo apt-get -y install ca-certificates curl')
    commands.append(f'sudo install -m 0755 -d /etc/apt/keyrings')
    commands.append(f'sudo curl -fsSL https://download.docker.com/linux/debian/gpg -o /etc/apt/keyrings/docker.asc')
    commands.append(f'sudo chmod a+r /etc/apt/keyrings/docker.asc')
    commands.append(f'echo "deb [arch=$(dpkg --print-architecture) signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/debian $(. /etc/os-release && echo "$VERSION_CODENAME") stable" | sudo tee /etc/apt/sources.list.d/docker.list > /dev/null')
    commands.append(f'sudo apt-get update')
    commands.append(f'sudo apt-get -y install docker-ce docker-ce-cli containerd.io docker-buildx-plugin docker-compose-plugin')
    commands.append(f'sudo groupadd docker')
    commands.append(f'sudo usermod -aG docker $USER')
    commands.append(f'sudo apt-get install -y jq')


for command in commands:
    print(f'Executing {command}')
    stdout, stderr = node.execute(command)
    
print('Done')

#### Check Management IP address (IPv4, IPv6)

In [ ]:
import ipaddress

def check_ip_version(ip_address):
    try:
        ip_object = ipaddress.ip_address(ip_address)
        if isinstance(ip_object, ipaddress.IPv4Address):
            return "IPv4"
        elif isinstance(ip_object, ipaddress.IPv6Address):
            return "IPv6"
    except ValueError:
        return "Invalid IP address"


### 1. Setup

<div class="alert alert-block alert-info">
<b>Testing - Declarations > </b> sdx-end-to-end-tests
</div>

In [ ]:

sdx_end_to_end_tests_repo = 'https://github.com/atlanticwave-sdx/sdx-end-to-end-tests.git'
sdx_end_to_end_tests_branch = 'main'
repo_dir = 'awsdx/sdx-end-to-end-tests'


### 2. Checkout

In [ ]:

commands1 = list()
commands1.append(f'[ ! -d ~/{repo_dir} ] && mkdir -p ~/{repo_dir}')
commands1.append(f'cd ~/{repo_dir} && git init && git remote add origin {sdx_end_to_end_tests_repo}')
commands1.append(f'cd ~/{repo_dir} && git fetch origin')
commands1.append(f'cd ~/{repo_dir} && git checkout {sdx_end_to_end_tests_branch}')

# Testing - customize for IPv6

# Add the following block to the docker-compose.yml
# Ref: https://docs.docker.com/reference/compose-file/networks/#the-default-network
#
#networks:
#   default:
#     name: sdxnet
#     enable_ipv6: true
#     ipam:
#       config:
#         - subnet: 2001:db8::/64

management_ip = node.get_management_ip()

if check_ip_version(management_ip) == 'IPv6':
    commands1.append(f'cd ~/{repo_dir} && echo -e "networks:\n  default:\n    name: sdxnet\n    enable_ipv6: true\n    ipam:\n      config:\n        - subnet: 2001:db8::/64" | tee -a docker-compose.yml > /dev/null')


for command in commands1:
    print(f'Executing {command}')
    stdout, stderr = node.execute(command)
    
print('Done')


### 3. Start Tests

In [ ]:

# Testing - start 

commands2 = list()
commands2.append(f'sudo systemctl start docker')
commands2.append(f'cd ~/{repo_dir} && docker compose up -d &> /tmp/docker-compose.log')
commands2.append(f'cd ~/{repo_dir} && docker compose ps -a')

# Testing - execute

commands3 = list()
commands3.append(f'cd ~/{repo_dir} && ./wait-mininet-ready.sh')
commands3.append(f'cd ~/{repo_dir} && docker compose exec -it mininet python3 -m pytest tests/ | tee /tmp/docker-compose-exec-pytest.out')
commands3.append(f'cd ~/{repo_dir} && docker compose logs sdx-controller -t > /tmp/docker-compose-logs-sdx-controller.out')


for command in commands2 + commands3:
    print(f'Executing {command}')
    stdout, stderr = node.execute(command)
    
print('Done')

In [ ]:
# Download

local_directory_path = 'docker-compose-logs-sdx-controller.out'
remote_directory_path = '/tmp/docker-compose-logs-sdx-controller.out'

node.download_file(local_directory_path,remote_directory_path)

print('Done')

### Verify docker image

Following steps are supplemental to verify the pulled docker image version wrt the github repo HEAD commit

In [ ]:
# Execute checks

print(f'--- Check the latest image metadata (revision) against the git repo commit')

commands1 = list()
commands1.append(f'docker image inspect awsdx/sdx-controller:latest | jq -r \'.[0].Config.Labels["org.opencontainers.image.revision"]\'')
commands1.append(f'git ls-remote https://github.com/atlanticwave-sdx/sdx-controller.git HEAD | cut -f1')

commands2 = list()
commands2.append(f'docker image inspect awsdx/sdx-lc:latest | jq -r \'.[0].Config.Labels["org.opencontainers.image.revision"]\'')
commands2.append(f'git ls-remote https://github.com/atlanticwave-sdx/sdx-lc.git HEAD | cut -f1')

for command in commands1 + commands2:
    print(f'Executing {command}')
    stdout, stderr = node.execute(command)
    
print('Done')


![Docker image metadata](./awsdx-e2e/docker-image-meta.png)

### Docker Hub Login (optional)
Create personal access token -> https://docs.docker.com/security/for-developers/access-tokens/

In [ ]:
# Docker Login - when necessary

docker_access_token = '<DOCKER_PERSONAL_ACCESS_TOKEN>'
docker_user_account = '<DOCKER_USER_ACCOUNT>'

commands = list()
#commands.append(f'echo {docker_access_token} | docker login --username {docker_user_account} --password-stdin')

for command in commands:
    print(f'Executing {command}')
    stdout, stderr = node.execute(command)
    
print('Done')

## Step 8: Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    ###slice.delete()
except Exception as e:
    print(f"Exception: {e}")